In [5]:
import os
import zipfile

def extract_zipfile(filename, dest):
    with open(filename, 'rb') as f:
        zf = zipfile.ZipFile(f)
        zf.extractall(dest)

def find_shps(d):
    total = 0
    for root, dirs, files in os.walk(d):
        for file_ in files:
            fn = os.path.join(root, file_)
            total += _count_file(fn)
    return total

In [7]:
def _count_file(fn):
    with open(fn, 'rb') as f:
        return _count_file_object(f)

def _count_file_object(f):
    # Note that this iterates on 'f'.
    # You *could* do 'return len(f.read())'
    # which would be faster but potentially memory 
    # inefficient and unrealistic in terms of this 
    # benchmark experiment. 
    total = 0
    for line in f:
        total += len(line)
    return total

In [8]:
src = "/media/arawlins/homedrive/arawlins/Documents/Final.Boundary.zip"
dest = "/media/arawlins/homedrive/arawlins/Documents/test/"

extract_zipfile(src, dest)
find_shps(dest)

881486

In [5]:
import geopandas as gp
import numpy as np
import pandas as pd
import xarray as xr
import regionmask

import json
import glob
import time
import cv2

from numpy import asarray
from scipy.spatial import ConvexHull

import cartopy.feature as cfeature
import cartopy.crs as ccrs

import shapely
from shapely.wkt import dumps, loads
from shapely.geometry import Polygon, mapping, shape
from shapely import affinity

from affine import Affine



In [31]:
geo_json = '''
{
    "type":"FeatureCollection",
    "features":[
        
        {
            "type":"Feature",
            "properties":{
                "FIREID":0,
                "FIRENAME":"Kilmore East",
                "AREATYPE":"Spotfire",
                "SOURCE":"API",
                "CONFIDENCE":"Very High",
                "AUTHOR":"Nic Gellie",
                "COMMENTS":"Digitised from post-fire aerial photography",
                "shape_area":1312855.9479,
                "shape_len":6045.9280357600001,
                "Area_ha":131.3,
                "Perim_km":6.0
            },
            "geometry":{
                "type":"MultiPolygon",
                "coordinates":[
                    [
                        [
                            [
                                145.473759276505746,
                                -37.711444444166929
                            ],
                            [
                                145.474224433339543,
                                -37.706250652018895
                            ],
                            [
                                145.48000011431634,
                                -37.704402044643601
                            ],
                            [
                                145.482019893331483,
                                -37.702077465098093
                            ],
                            [
                                145.485424253951521,
                                -37.702192482625897
                            ],
                            [
                                145.484212991212161,
                                -37.70003999960899
                            ],
                            [
                                145.485954962710139,
                                -37.698139159903093
                            ],
                            [
                                145.489278209274062,
                                -37.702241933715939
                            ],
                            [
                                145.49263623843774,
                                -37.700219232246781
                            ],
                            [
                                145.49405150206664,
                                -37.701163317169602
                            ],
                            [
                                145.494914637603671,
                                -37.7028126432335
                            ],
                            [
                                145.488526430951197,
                                -37.708358119288569
                            ],
                            [
                                145.483908653685063,
                                -37.710578192600657
                            ],
                            [
                                145.473759276505746,
                                -37.711444444166929
                            ]
                        ]
                    ]
                ]
            }
        }
    ]
}'''
import json    
geo_json = json.loads(geo_json)

In [33]:
selections = list()
count = 0
numbers = []
names = []
abbrevs = []
if geo_json["type"] == "FeatureCollection":
    print("Found a feature collection...")
    for p in geo_json["features"]:

        print("Found a Feature.")
        if p["geometry"]["type"] == "Polygon":
            print("Found Polygon #%s" % count)
            points = p["geometry"]["coordinates"]
            s = shapely.geometry.Polygon(np.asarray(*points))
            selections.append(s)
            numbers.append(count)
            names.append("Selection_%s" % count)
            abbrevs.append("SEL_%s" % count)
            count += 1

        if p["geometry"]["type"] == "MultiPolygon":
            print("Found MultiPolygon #%s" % count)
            points = p["geometry"]["coordinates"]
            print(points)
#             s = [x.buffer(0) for x in shape(points).buffer(0).geoms]
            pointcloud = np.asarray(*points)
            print(pointcloud.ndim)
            if pointcloud.ndim > 2 and pointcloud.shape[0] != 1:
                s = shape(points).buffer
            else:
                s = shape(points)
            selections.append(s)
            numbers.append(count)
            names.append("Selection_%s" % count)
            abbrevs.append("SEL_%s" % count)
            count += 1

print("With %s Polygons." % count)
print("numbers: %s" % numbers)
print("names: %s" % names)
print("abbrevs: %s" % abbrevs)
print("selections: %s" % selections)

Found a feature collection...
Found a Feature.
Found MultiPolygon #0
[[[[145.47375927650575, -37.71144444416693], [145.47422443333954, -37.706250652018895], [145.48000011431634, -37.7044020446436], [145.48201989333148, -37.70207746509809], [145.48542425395152, -37.7021924826259], [145.48421299121216, -37.70003999960899], [145.48595496271014, -37.69813915990309], [145.48927820927406, -37.70224193371594], [145.49263623843774, -37.70021923224678], [145.49405150206664, -37.7011633171696], [145.49491463760367, -37.7028126432335], [145.4885264309512, -37.70835811928857], [145.48390865368506, -37.71057819260066], [145.47375927650575, -37.71144444416693]]]]
3


AttributeError: 'list' object has no attribute 'get'

In [46]:
if geo_json["type"] == "FeatureCollection":
    print("Found a feature collection...")
    for p in geo_json["features"]:
        print("Found a", p['type'])
        print(p['properties'])
#         print(p['geometry'])
        print("Found a", p['geometry']['type'])
        coords = p['geometry']['coordinates']
        g_type = p['geometry']['type']
        
        if g_type == 'MultiPolygon':
            s = shapely.geometry.MultiPolygon(coords)
            
        if g_type == 'Polygon':
            s = shapely.geometry.Polygon(coords)

Found a feature collection...
Found a Feature
{'FIREID': 0, 'FIRENAME': 'Kilmore East', 'AREATYPE': 'Spotfire', 'SOURCE': 'API', 'CONFIDENCE': 'Very High', 'AUTHOR': 'Nic Gellie', 'COMMENTS': 'Digitised from post-fire aerial photography', 'shape_area': 1312855.9479, 'shape_len': 6045.92803576, 'Area_ha': 131.3, 'Perim_km': 6.0}
Found a MultiPolygon


AttributeError: 'list' object has no attribute 'buffer'

In [52]:
for p in geo_json["features"]:
    s = shape(p["geometry"])
    print(s)

MULTIPOLYGON (((145.4737592765057 -37.71144444416693, 145.4742244333395 -37.70625065201889, 145.4800001143163 -37.7044020446436, 145.4820198933315 -37.70207746509809, 145.4854242539515 -37.7021924826259, 145.4842129912122 -37.70003999960899, 145.4859549627101 -37.69813915990309, 145.4892782092741 -37.70224193371594, 145.4926362384377 -37.70021923224678, 145.4940515020666 -37.7011633171696, 145.4949146376037 -37.7028126432335, 145.4885264309512 -37.70835811928857, 145.4839086536851 -37.71057819260066, 145.4737592765057 -37.71144444416693)))


In [32]:
geo_json

{'features': [{'geometry': {'coordinates': [[[[145.47375927650575,
        -37.71144444416693],
       [145.47422443333954, -37.706250652018895],
       [145.48000011431634, -37.7044020446436],
       [145.48201989333148, -37.70207746509809],
       [145.48542425395152, -37.7021924826259],
       [145.48421299121216, -37.70003999960899],
       [145.48595496271014, -37.69813915990309],
       [145.48927820927406, -37.70224193371594],
       [145.49263623843774, -37.70021923224678],
       [145.49405150206664, -37.7011633171696],
       [145.49491463760367, -37.7028126432335],
       [145.4885264309512, -37.70835811928857],
       [145.48390865368506, -37.71057819260066],
       [145.47375927650575, -37.71144444416693]]]],
    'type': 'MultiPolygon'},
   'properties': {'AREATYPE': 'Spotfire',
    'AUTHOR': 'Nic Gellie',
    'Area_ha': 131.3,
    'COMMENTS': 'Digitised from post-fire aerial photography',
    'CONFIDENCE': 'Very High',
    'FIREID': 0,
    'FIRENAME': 'Kilmore East',
    